In [1]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import seaborn as sns
import pathlib
import sklearn
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import math
import codecs
import unidecode
import re

# Data preparation 👨‍🍳
## Get lyrics data

In [4]:
lyrics =codecs.open('all_lyrics.txt', 'r', 'utf-8').read()

## Reduce vocabulary by replacing character
Only keep lowcase letters and space to make learning easier.

In [5]:
simplified_lyrics=unidecode.unidecode(lyrics.lower())
simplified_lyrics = simplified_lyrics.replace('!','')
simplified_lyrics = simplified_lyrics.replace('"','')
simplified_lyrics = simplified_lyrics.replace("'",'')
simplified_lyrics = simplified_lyrics.replace('+','')
simplified_lyrics = simplified_lyrics.replace('-','')
simplified_lyrics = simplified_lyrics.replace('<','')
simplified_lyrics = simplified_lyrics.replace('>','')
simplified_lyrics = simplified_lyrics.replace('.','')
simplified_lyrics = re.sub(r'\d', '#', simplified_lyrics)

## Check vocabulary

In [6]:
vocab=sorted(set(simplified_lyrics))
print(vocab)

[' ', '#', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Tokenization
### Dictionnaries of conversion
Dictionnaries to interchange integers and characters.

In [7]:
char_to_int = dict((c, i) for i, c in enumerate(vocab))
int_to_char = dict((i, c) for i, c in enumerate(vocab))

### Tokenize text
Transform characters to integers.

In [13]:
num_lyrics=[char_to_int[c] for c in simplified_lyrics]

print(simplified_lyrics[:10], ' -> ', num_lyrics[:10])

maintenant  ->  [14, 2, 10, 15, 21, 6, 15, 2, 15, 21]


## Make sequences with dataset

In [9]:
seq_length = 80
dataX = []
dataY = []

for i in range(0, len(num_lyrics) - seq_length, 1):
    seq_in = simplified_lyrics[i:i + seq_length]
    seq_out = simplified_lyrics[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

# Create dataset 📀
Large bacth size to be able to learn on all dataset due to lack of ressources, moreover it increases the perfomrence for text generation.

Don't shuffle dataset to keep consistency of the data.

<b> Information : http://ceur-ws.org/Vol-3027/paper108.pdf

In [ ]:
BATCH_SIZE = 2048

dataset = tf.data.Dataset.from_tensor_slices((dataX, dataY))

dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

# Define model 💡
Use of Bidirectional LSTM layers to learn from sequences.

In [10]:
vocab_size=len(vocab)
vector_size=256

model=keras.Sequential([
    layers.Embedding(vocab_size, vector_size, input_length=seq_length),
    layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
    layers.Bidirectional(keras.layers.LSTM(128, return_sequences=False)),
    layers.Dense(vocab_size, activation='softmax')      
])

model.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer='adam')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 256)           7168      
                                                                 
 bidirectional (Bidirectiona  (None, 80, 256)          394240    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 28)                7196      
                                                                 
Total params: 802,844
Trainable params: 802,844
Non-trainable params: 0
_________________________________________________________________


# Train model ⏳

In [10]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=15,
    min_delta=0.001,
    restore_best_weights=True,
);


model.fit(dataset, epochs=200, batch_size=BATCH_SIZE,callbacks=[early_stopping])
model.save("NekIA.h5")

Epoch 1/200


2022-10-30 16:30:55.245858: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-30 16:30:56.633638: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


531/531 [==============================] - 110s 193ms/step - loss: 2.2231
Epoch 2/200
531/531 [==============================] - 102s 193ms/step - loss: 1.8277
Epoch 3/200
531/531 [==============================] - 103s 194ms/step - loss: 1.6977
Epoch 4/200
531/531 [==============================] - 103s 194ms/step - loss: 1.6210
Epoch 5/200
531/531 [==============================] - 103s 194ms/step - loss: 1.5643
Epoch 6/200
531/531 [==============================] - 103s 194ms/step - loss: 1.5213
Epoch 7/200
531/531 [==============================] - 103s 194ms/step - loss: 1.4871
Epoch 8/200
531/531 [==============================] - 103s 194ms/step - loss: 1.4585
Epoch 9/200
531/531 [==============================] - 102s 193ms/step - loss: 1.4337
Epoch 10/200
531/531 [==============================] - 102s 193ms/step - loss: 1.4119
Epoch 11/200
531/531 [==============================] - 102s 193ms/step - loss: 1.3925
Epoch 12/200
531/531 [==============================] - 103s 194

# Make a prediction 🎤

The first sentence must be in the same format as defined, with only 80 lowercase letters and a space.

In [12]:
model=keras.models.load_model('NekIA.h5')
start = "avant davoir le respect essaye davoir le mental mais mauvaise mauvaise graine so"

pattern = [char_to_int[c] for c in start]


print('START : ',''.join([int_to_char[i] for i in pattern]))
print('PREDICTION : ', end='')

for i in range(80):
    x = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(result, end = '')
    pattern.append(index)
    pattern = pattern[1:len(pattern)]


START :  avant davoir le respect essaye davoir le mental mais mauvaise mauvaise graine so
PREDICTION : us le maitre de ta mere jai les crocs doux soirees en lair est arrete de tourner